## Importación de librerías

In [73]:
import pandas as pd
import os
import glob
import json
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder

## Configuración de variables globales

In [74]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
DATASETS_PATH = SETUP_JSON['datasets_path'] # Ruta de la carpeta de los datasets,
DATASETS_FOLDER = os.path.join(os.getcwd(), DATASETS_PATH) # Carpeta de los datasets,
DATASETS = glob.glob(os.path.join(DATASETS_FOLDER, '*.csv')) # Lista de los datasets
OUTPUT_CSV = SETUP_JSON['dataset_csv'] # Fichero CSV de salida
OUTPUT_PARQUET = SETUP_JSON['dataset_parquet'] # Fichero Parquet de salida
TAKE_FULL_DATASET = SETUP_JSON['take_full_dataset'] # Obtener el dataset completo o una muestra
NA_VAL = SETUP_JSON['navalues'] # Valores a considerar como NaN
balances = SETUP_JSON['balances'] # Balances a considerar

## Análisis de los datasets

Obtener todos los encabezados diferentes

In [75]:
unique_headers = set()
for dataset in DATASETS:
    df_dtypes = pd.read_csv(dataset, nrows=1).dtypes.keys()
    unique_headers.add(tuple(df_dtypes))
for header in unique_headers:
    print(header)

('Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Fwd Byts/b Avg', 'F

Separar los grupos de datasets por encabezados diferentes

In [76]:
datasets_by_header = {}
for header in unique_headers:
    datasets_group = []
    for dataset in DATASETS:
        df_dtypes = pd.read_csv(dataset, nrows=1).dtypes.keys()
        if tuple(df_dtypes) == header:
            datasets_group.append(dataset)
    datasets_by_header[header] = datasets_group
    print(f"Grupo de datasets:")
    for dataset in datasets_group:
        print(dataset)
    print(f"Número de columnas del grupo de datasets: {len(header)}")
    print()

Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-20-2018.csv
Número de columnas del grupo de datasets: 84

Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv
Número de columnas del grupo de datasets: 80



Obtener etiquetas diferentes encontradas en cada grupo de datasets

In [77]:
datasets_by_num_labels = {}

for datasets_group in datasets_by_header.values():
    unique_labels = set()
    for dataset in datasets_group:
        all_labels = list(pd.read_csv(dataset, usecols=["Label"])["Label"])
        for label in all_labels:
            unique_labels.add(label)
    datasets_by_num_labels[tuple(datasets_group)] = len(unique_labels)
    print(f"Grupo de datasets:")
    for dataset in datasets_group:
        print(dataset)
    print(f"Etiquetas diferentes encontradas: ", unique_labels)


Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-20-2018.csv
Etiquetas diferentes encontradas:  {'Benign', 'DDoS attacks-LOIC-HTTP'}
Grupo de datasets:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv
Etiquetas diferentes encontradas:  {'DDOS attack-HOIC', 'DoS attacks-SlowHTTPTest', 'Brute Force -XS

Seleccionar el grupo de datasets que abarca más etiquetas diferentes

In [78]:
for datasets_group, num_labels in datasets_by_num_labels.items():
    if (num_labels) == max(datasets_by_num_labels.values()):
        datasets_selected = datasets_group
print(f"Selección de datasets a procesar:")
for dataset in datasets_selected:
    print(dataset)

Selección de datasets a procesar:
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-16-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-21-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-22-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-23-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv
c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv


## Combinar datasets

Unimos todos los datasets seleccionados en un sólo dataframe

In [79]:
df = pd.DataFrame()
if TAKE_FULL_DATASET:
    for dataset in datasets_selected:
        current_df = pd.read_csv(dataset, sep=',', low_memory=False, na_values=NA_VAL) 
        df = pd.concat([df, current_df], ignore_index=True)
        print(f"Dataset concatenado: {dataset}, Dimensiones: {current_df.shape}")

else:
    for dataset in datasets_selected:
        current_df = pd.read_csv(dataset, sep=',', low_memory=False, na_values=NA_VAL)
        
        # Agregar una columna auxiliar que cuenta la aparición de cada Label
        current_df["Label_Count"] = current_df.groupby("Label").cumcount() + 1

        # Filtrar solo las primeras N apariciones de cada Label
        copy_df = current_df[current_df.apply(lambda row: row["Label_Count"] <= balances.get(row["Label"], 0), axis=1)]

        # Eliminar columna auxiliar antes de concatenar
        copy_df.drop(columns=["Label_Count"], inplace=True)

        df = pd.concat([df, copy_df], ignore_index=True)

        print(f"Dataset concatenado: {dataset}, Dimensiones: {copy_df.shape}")


C:\Users\isard\AppData\Local\Temp\ipykernel_2696\1079085574.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df.drop(columns=["Label_Count"], inplace=True)


Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-14-2018.csv, Dimensiones: (121000, 80)


C:\Users\isard\AppData\Local\Temp\ipykernel_2696\1079085574.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df.drop(columns=["Label_Count"], inplace=True)


Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-15-2018.csv, Dimensiones: (10000, 80)


C:\Users\isard\AppData\Local\Temp\ipykernel_2696\1079085574.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df.drop(columns=["Label_Count"], inplace=True)


...

C:\Users\isard\AppData\Local\Temp\ipykernel_2696\1079085574.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df.drop(columns=["Label_Count"], inplace=True)


Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\02-28-2018.csv, Dimensiones: (11000, 80)


C:\Users\isard\AppData\Local\Temp\ipykernel_2696\1079085574.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df.drop(columns=["Label_Count"], inplace=True)


Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-01-2018.csv, Dimensiones: (11000, 80)
Dataset concatenado: c:\Users\isard\Desktop\AI-for-Traffic-network-classify\datasets\03-02-2018.csv, Dimensiones: (11000, 80)


C:\Users\isard\AppData\Local\Temp\ipykernel_2696\1079085574.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copy_df.drop(columns=["Label_Count"], inplace=True)


In [80]:
df['Label'].value_counts()

Label
FTP-BruteForce              60000
SSH-Bruteforce              60000
Infilteration               20000
DDOS attack-HOIC            20000
Bot                         10000
Benign                       9000
DoS attacks-GoldenEye        6000
DoS attacks-Slowloris        3000
DoS attacks-SlowHTTPTest     3000
DoS attacks-Hulk             3000
DDOS attack-LOIC-UDP         1730
Brute Force -Web              611
Brute Force -XSS              230
SQL Injection                  87
Name: count, dtype: int64

In [81]:
df.shape

(196658, 80)

## Análisis

In [82]:
# Verificar si hay encabezados duplicados
df_headers = df[df['Label'] == 'Label']
df_headers

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label


In [83]:
# Ver la cantidad de filas con valores faltantes
df.isna().any(axis=1).value_counts()

False    196459
True        199
Name: count, dtype: int64

In [84]:
# Ver etiquetas únicas del dataframe
unique_labels = df['Label'].unique()
print(f"{len(unique_labels)} etiquetas únicas: {unique_labels}")

14 etiquetas únicas: ['Benign' 'FTP-BruteForce' 'SSH-Bruteforce' 'DoS attacks-GoldenEye'
 'DoS attacks-Slowloris' 'DoS attacks-SlowHTTPTest' 'DoS attacks-Hulk'
 'DDOS attack-LOIC-UDP' 'DDOS attack-HOIC' 'Brute Force -Web'
 'Brute Force -XSS' 'SQL Injection' 'Infilteration' 'Bot']


In [85]:
# Ver los tipos de datos de las columnas
df.dtypes

Dst Port         object
Protocol         object
Timestamp        object
Flow Duration    object
Tot Fwd Pkts     object
                  ...  
Idle Mean        object
Idle Std         object
Idle Max         object
Idle Min         object
Label            object
Length: 80, dtype: object

## Limpieza

In [86]:
# Eliminar encabezados repetidos
df = df[df['Label'] != 'Label']
df.shape

(196658, 80)

In [87]:
# Eliminar los valores faltantes
df = df.dropna()
df.shape

(196459, 80)

In [88]:
df.reset_index(drop=True, inplace=True)

Crear un json con todos los valores de etiquetas mapeados a números

In [89]:
# Obtener los valores únicos de la columna 'Label'
unique_labels = df['Label'].unique()

# Crear un diccionario mapeando los valores únicos a números
label_mapping = {label: int(idx) for idx, label in enumerate(unique_labels)}
# Invertir el diccionario (intercambiar claves y valores)
inverted_mapping = {v: k for k, v in label_mapping.items()}

# Guardar el diccionario invertido como un archivo JSON
with open('labels.json', 'w') as json_file:
    json.dump(inverted_mapping, json_file, indent=4)

print("Archivo JSON creado con el mapeo de etiquetas")

Archivo JSON creado con el mapeo de etiquetas


## Normalización

Convertir etiquetas a números

In [90]:
# # Transformar cada valor de la columna 'Label' a su equivalente código
# le = LabelEncoder()
# # Crear nueva columna con el código de cada etiqueta
# df['Label Code'] = le.fit_transform(df['Label'])

    # Transformar cada valor de la columna 'Label' a su equivalente según el mapeo de labels
df['Label Code'] = df['Label'].map(label_mapping)

Asignar tipo correspondiente a las columnas

In [91]:
# Asignar formato decimal a columnas numéricas
for col, dtype in df.dtypes.to_dict().items():
    temp_col = df[col].dropna()
    numeric_col = pd.to_numeric(temp_col, errors='coerce').notna().all()
    if numeric_col:
        df[col] = pd.to_numeric(temp_col, errors='coerce').astype('float64')
        
# Asignar formato entero a columnas que no contienen decimales
df["Protocol"] = df["Protocol"].astype(int)
df["Dst Port"] = df["Dst Port"].astype(int)
df["Label Code"] = df["Label Code"].astype(int)

# Conversión de formato de timestamp a formato legible para el modelo
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S', errors='coerce') # Convertimos la timestamp a formato fecha de pandas
df['Timestamp'] = np.sin(2*math.pi*df['Timestamp'].dt.hour/24)  # Convertimos la fecha en un ciclo legible para el modelo

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196459 entries, 0 to 196458
Data columns (total 81 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           196459 non-null  int64  
 1   Protocol           196459 non-null  int64  
 2   Timestamp          196459 non-null  float64
 3   Flow Duration      196459 non-null  float64
 4   Tot Fwd Pkts       196459 non-null  float64
 5   Tot Bwd Pkts       196459 non-null  float64
 6   TotLen Fwd Pkts    196459 non-null  float64
 7   TotLen Bwd Pkts    196459 non-null  float64
 8   Fwd Pkt Len Max    196459 non-null  float64
 9   Fwd Pkt Len Min    196459 non-null  float64
 10  Fwd Pkt Len Mean   196459 non-null  float64
 11  Fwd Pkt Len Std    196459 non-null  float64
 12  Bwd Pkt Len Max    196459 non-null  float64
 13  Bwd Pkt Len Min    196459 non-null  float64
 14  Bwd Pkt Len Mean   196459 non-null  float64
 15  Bwd Pkt Len Std    196459 non-null  float64
 16  Fl

In [93]:
cols_with_inf = df.drop(columns=['Label']).columns[np.isinf(df.drop(columns=['Label'])).any()]
cols_with_inf

Index(['Flow Byts/s', 'Flow Pkts/s'], dtype='object')

In [94]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label Code
0,0,0,0.866025,112641719.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56320859.5,139.300036,56320958.0,56320761.0,Benign,0
1,0,0,0.866025,112641466.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56320733.0,114.551299,56320814.0,56320652.0,Benign,0
2,0,0,0.866025,112638623.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,56319311.5,301.934596,56319525.0,56319098.0,Benign,0
3,22,6,0.866025,6453966.0,15.0,10.0,1239.0,2273.0,744.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign,0
4,22,6,0.866025,8804066.0,14.0,11.0,1143.0,2209.0,744.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,Benign,0


In [95]:
df.tail()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label Code
196454,8080,6,0.258819,447.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,13
196455,8080,6,0.258819,10917.0,3.0,4.0,326.0,129.0,326.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,13
196456,8080,6,0.258819,495.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,13
196457,8080,6,0.258819,11468.0,3.0,4.0,326.0,129.0,326.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,13
196458,8080,6,0.258819,484.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Bot,13


## Exportación del dataframe final en formato CSV y Parquet

In [96]:
# Guardar en CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"Archivo CSV guardado como {OUTPUT_CSV}")

df.to_parquet(OUTPUT_PARQUET, index=False)
print(f"Archivo parquet guardado como {OUTPUT_PARQUET}")

Archivo CSV guardado como clean_dataset.csv
Archivo parquet guardado como clean_dataset.parquet
